# Catmull-Rom interpolation

In [1]:
import numpy

def get_items_count(items):
    # If the lengths of the 'x' and 'y' lists are equivalent (they SHOULD be)
    if len(items['x']) == len(items['y']):
        # Return the length of one of them
        return len(items['x'])
    # In any other case, print and return an error
    else:
        print('error: ' + str(items))
        return 'error'


# Returns number of days between the target date and the 31st of
# december of 2000. Given the data, the least integer will be 0
def date_to_days(date):
    return (date - datetime.date(2000, 12, 31)).days


# Returns a date
def days_to_date(day):
    return datetime.date(2000, 12, 31) + datetime.timedelta(days=day)


# Returns the linear interpolation of the items
def get_linear_interpolation(items):
    # Get the 'x' and 'y' values of the first and last point
    #firstX = date_to_days(items['x'][0])
    #lastX = date_to_days(items['x'][len(items) - 1])
    firstX = items['x'][0]
    lastX = items['x'][len(items) - 1]

    firstY = items['y'][1]
    lastY = items['y'][len(items) - 1]

    # Calculate the slope and the Y-intercept
    slope = (lastY - firstY) / (lastX - firstX)
    yIntercept = firstY - firstX * slope

    # Create the points, from the first one to the last one, using the slope
    points = []
    for x in range(firstX, lastX):
        y = yIntercept + slope * x

        points.append([x, y])

    return points


# Given the nature of the experiment, there are points on the beginning
# and end of the graph that have no values
def add_nils_to_interpolation(points):
    # Get the first and last days of the interpolation
    firstDay = points[0][0]
    lastDay = points[len(points) - 1][0]

    for day in range(0, firstDay):
        points = [[firstDay - day - 1, 'nil']] + points  # Preppend

    for day in range(lastDay + 1, daysMax):
        points = points + [[day, 'nil']]  # Append

    return points


# Returns the items as a list of points: [{x0, y0}, {x1, y1}, {x2, y2}, ...]
# integer_dates: (Boolean) True if the dates have to be integers
def get_points(items, integer_dates=False):
    points = []

    # For each item, create and add a new point
    for i in range(len(items['x'])):

        y = items['y'][i]

        if integer_dates:
            x = date_to_days(items['x'][i])
        else:
            x = items['x'][i]

        points.append([x, y])

    return points


def add_border_points(points):
    diff_resolution = 10

    # Get the change in x and y between the first and second coordinates.
    dx = points[1][0] - points[0][0]
    dy = points[1][1] - points[0][1]

    # Then using the change, extrapolate backwards to find a control point.
    x1 = points[0][0] - (dx / diff_resolution)
    y1 = points[0][1] - (dy / diff_resolution)

    # Create the start point from the extrapolated values.
    start = [x1, y1]

    # Repeat for the end control point.
    n = len(points) - 1
    dx = points[n][0] - points[n - 1][0]
    dy = points[n][1] - points[n - 1][1]

    xn = points[n][0] + (dx / diff_resolution)
    yn = points[n][1] + (dy / diff_resolution)
    end = [xn, yn]

    # insert the start control point at the start of the points list.
    final_points = [start] + points

    # append the end control ponit to the end of the points list.
    final_points.append(end)

    return final_points


# Source: Wikipedia, https://en.wikipedia.org/wiki/Centripetal_Catmull%E2%80%93Rom_spline
def catmull_rom_spline(P0, P1, P2, P3, nPoints=10):
    """
    P0, P1, P2, and P3 should be (x,y) point pairs that define the Catmull-Rom spline.
    nPoints is the number of points to include in this curve segment.
    """
    # Convert the points to numpy so that we can do array multiplication
    P0, P1, P2, P3 = map(numpy.array, [P0, P1, P2, P3])

    # Calculate t0 to t4
    alpha = 0.5

    def tj(ti, Pi, Pj):
        xi, yi = Pi
        xj, yj = Pj
        return (((xj - xi) ** 2 + (yj - yi) ** 2) ** 0.5) ** alpha + ti

    t0 = 0
    t1 = tj(t0, P0, P1)
    t2 = tj(t1, P1, P2)
    t3 = tj(t2, P2, P3)

    # nPoints = numpy.sqrt ( numpy.power(P2[0]-P1[0], 2) + numpy.power(P2[1]-P1[1], 2) )

    # Only calculate points between P1 and P2
    t = numpy.linspace(t1, t2, nPoints)

    # Reshape so that we can multiply by the points P0 to P3
    # and get a point for each value of t.
    t = t.reshape(len(t), 1)

    A1 = (t1 - t) / (t1 - t0) * P0 + (t - t0) / (t1 - t0) * P1
    A2 = (t2 - t) / (t2 - t1) * P1 + (t - t1) / (t2 - t1) * P2
    A3 = (t3 - t) / (t3 - t2) * P2 + (t - t2) / (t3 - t2) * P3

    B1 = (t2 - t) / (t2 - t0) * A1 + (t - t0) / (t2 - t0) * A2
    B2 = (t3 - t) / (t3 - t1) * A2 + (t - t1) / (t3 - t1) * A3

    C = (t2 - t) / (t2 - t1) * B1 + (t - t1) / (t2 - t1) * B2
    return C


# Source: Wikipedia, https://en.wikipedia.org/wiki/Centripetal_Catmull%E2%80%93Rom_spline
def catmull_rom_chain(points, nPointsCatmullRom=10):
    """
    Calculate Catmull Rom for a chain of points and return the combined curve.
    """
    points = add_border_points(points)

    size = len(points)

    # The curve C will contain an array of (x,y) points.
    C = []
    for i in range(size - 3):
        c = catmull_rom_spline(points[i], points[i + 1], points[i + 2], points[i + 3], nPointsCatmullRom)

        C.extend(c)

    return C


# Returns the RAW Centripetal Catmull-Rom points of the current Response
def get_raw_catmull_rom(items):
    # Thousands of Catmull-Rom points because we want great smoothness
    nPoints = 2000

    return catmull_rom_chain(get_points(items, False), nPoints)


# Returns the index of the point which has the closest X value to the targetValue
# The search is started in startSearchOn
def find_closest_x(points, targetValue, startSearchOn=0):
    minimumDistance = 999999999  # Very large number
    indexOfMinimum = -1  # Index where the minimum distance to the target is found

    # Search on the entire length of the
    for i in range(startSearchOn, len(points)):
        distance = abs(targetValue - points[i][0])

        # If the new distance is smaller than the current minimum distance, save it
        if distance <= minimumDistance:
            minimumDistance = distance
            indexOfMinimum = i
        else:
            # Given that the distance is described by a convex function,
            # once the distance increases, then it means the minimum has
            # been passed

            if distance - minimumDistance > 10:
                break

    # Return the index where the minimum distance was found
    return indexOfMinimum


# Returns the Catmull interpolation. Only to be used when the
# number of items is three or more
def catmull_rom(items):
    spline = get_raw_catmull_rom(items)

    # Get the 'x' value for the first and last point of the spline
    firstX = int(round(spline[0][0]))
    lastX = int(round(spline[len(spline) - 1][0]))

    searchIndex = 0

    filteredPoints = []

    # For each of the x values that need to be filled with a value...
    for x in range(firstX, lastX + 1):
        # Search for the point that it closest to it, starting with the search
        # at the searchIndex
        searchIndex = find_closest_x(spline, x, searchIndex)

        # Add the found point to the filteredPoints list
        filteredPoints.append([x, spline[searchIndex][1]])

    return filteredPoints


# Returns the FILTERED Centripetal Catmull-Rom points of the current Response
# By filtered, it means there is one point per day.
# This returns an array of arrays, in this form: [[x0, y0], [x1, y1], ...]
def get_catmull_rom(items, addNils=False, useDates=False):
    # Linear interpolation because there are only two points
    if get_items_count(items) == 2:
        points = get_linear_interpolation(items)
    # Length larger than 2, because the minimum length is 2
    else:
        points = catmull_rom(items)

    # Should nil values be added to the array?
    if addNils:
        points = self.add_nils_to_interpolation(points)

    # Transform the dates
    if useDates:
        for p in points:
            p[0] = days_to_date(p[0])

    return points

In [2]:
# Example of use:

items = {'x': [1, 10, 20], 
         'y': [0, 10, 0]}

print(get_catmull_rom(items))

[[1, 0.0], [2, 1.3628748884699755], [3, 2.865616382864006], [4, 4.3494380055768698], [5, 5.7446587887752649], [6, 7.0309553171340173], [7, 8.159752216475157], [8, 9.0813808093693282], [9, 9.7310617263114771], [10, 10.0], [11, 9.790087044096845], [12, 9.2293591670070381], [13, 8.4301287963861], [14, 7.45418049394367], [15, 6.3477482524831945], [16, 5.1408745439162224], [17, 3.8582966335706845], [18, 2.531358466818916], [19, 1.2042294563885068], [20, 0.0]]


### Importing real-world data

<table>
    <tr>
        <td>**Temperature**</td><td>already has one datapoint per day</td><td>✔</td>
    </tr>
    <tr>
        <td>**Rain**</td><td>same</td><td>✔</td>
    </tr>
    <tr>
        <td>**Sales**</td><td>Data is monthly</td><td>✔</td>
    </tr>
    <tr>
        <td>**Gym**</td><td>Data is weekly (full date though)</td><td>✔</td>
    </tr>
    <tr>
        <td>**Facebook**</td><td>Data is by quarters</td><td>✔</td>
    </tr>
    <tr>
        <td>**Salary**</td><td>Data is yearly</td>
    </tr>
</table>

All the data used was from 2013 to 2016 (including borders), except for Facebook, for which the interval 2010-2013 was used.

In [3]:
import pandas
from datetime import datetime

In [4]:
first = datetime(2013, 1, 1)

#Gets the days between X and Jan 1st 2013
def distance(datetime1, datetime2 = first):
    return (datetime1 - datetime2).days + 1

# The following functions are for monthly, quarterly, yearly, and full dates calculations.

def get_days_m(year, month):
    d = datetime(year, month, 1)
    return distance(d)

def get_days_q(year, quarter, d2=datetime(2010, 1, 1)):
    d = datetime(year, ((quarter-1)*3 + 1) , 1)
    return distance(d, d2)

def get_days_y(year):
    d = datetime(year, 1, 1)
    return distance(d)

def get_days_full(year, week, day):
    d = datetime(year, week, day)
    return distance(d)

### Rain

In [5]:
rain = pandas.read_csv('data/real-world/filtered_and_transformed/weather.csv')



### Sales

In [6]:
sales = pandas.read_csv('data/real-world/filtered_and_transformed/sales.csv')

for index, row in sales.iterrows():
    sales.loc[index, 'days_n'] = get_days_m(int(row['Year']), int(row['Month']))

sales.head()

,id,Year,Month,Value,days_n
0,1,2013,1,87.5,1.0
1,2,2013,2,90.1,32.0
2,3,2013,3,93.6,60.0
3,4,2013,4,92.6,91.0
4,5,2013,5,96.9,121.0


### Gym

In [7]:
gym = pandas.read_csv('data/real-world/filtered_and_transformed/gym.csv')

for index, row in gym.iterrows():
    gym.loc[index, 'days_n'] = get_days_full(int(row['year']), int(row['month']), int(row['day']))
    
gym.head()

,id,week,gym_uk,day,month,year,days_n
0,1,06-01-13,56,6,1,2013,6.0
1,2,13-01-13,48,13,1,2013,13.0
2,3,20-01-13,47,20,1,2013,20.0
3,4,27-01-13,51,27,1,2013,27.0
4,5,03-02-13,48,3,2,2013,34.0


### Facebook

In [8]:
facebook = pandas.read_csv('data/real-world/filtered_and_transformed/facebook.csv')

for index, row in facebook.iterrows():
    facebook.loc[index, 'days_n'] = get_days_q(int(row['year']), int(row['quarter_n']))
    
facebook.head()

,id,year,quarter,users_millions,quarter_n,days_n
0,1,2010,q1,130,1,1.0
1,2,2010,q2,137,2,91.0
2,3,2010,q3,144,3,182.0
3,4,2010,q4,154,4,274.0
4,5,2011,q1,163,1,366.0


### Salary

In [9]:
salary = pandas.read_csv('data/real-world/filtered_and_transformed/salary.csv')

for index, row in salary.iterrows():
    salary.loc[index, 'days_n'] = get_days_y(int(row['year']))
    
salary.head()

,id,year_n,age_us,age_uk,usd_per_year,year,days_n
0,1,4,26,25,39000,2013,1.0
1,2,5,27,26,41000,2014,366.0
2,3,6,28,27,44000,2015,731.0
3,4,7,29,28,45000,2016,1096.0
4,5,8,30,29,47000,2017,1462.0


### Interpolation

In [10]:
def get_interpolation(real_world_data, str_y):
    return get_catmull_rom({'x': real_world_data['days_n'], 'y': real_world_data[str_y]})

In [11]:
sales_i = get_interpolation(sales, 'Value')
gym_i = get_interpolation(gym, 'gym_uk')
facebook_i = get_interpolation(facebook, 'users_millions')
salary_i = get_interpolation(salary, 'usd_per_year')